In [1]:
import pandas as pd
import numpy as np
import itertools, re
from nltk.stem import StemmerI
from nltk.metrics import edit_distance
import nltk

In [2]:
class IndianNameStemmer(StemmerI):
    def stem(self, token):
        newtup = list()
        for i in token:
            i = i.lower()
            i = re.sub(r'\([^)]*\)', '', i).strip()
            i = re.sub(r'\[[^)]*\]', '', i).strip()
            i = re.sub(r'(^\w{1,3}\. ?)', r'', i)
            i = i.split('.')[0]
            i = i.split('-')[0]
            i = re.sub(r'[^a-zA-Z]', '', i)
            i = re.sub(r'(\w)\1+', r'\1', i)
            i = i[:-2] if i.endswith(('bi', 'ji', 'so')) else i
            i = i[:-3] if i.endswith(('bai', 'ben', 'beg', 'tai', 'sab', 'rao', 'rav', 'kha', 'lal', 'dev', 'deo')) else i
            i = i[:-4] if i.endswith(('bhai', 'bhau', 'sing', 'devi')) else i
            i = i[:-5] if i.endswith(('saheb', 'singh', 'shing', 'kumar')) else i
            for r in (
                ("tha", "ta"), ("krish", "krush"), ("gi", "ji"), ("bh", "b"),
                ("yu", "u"), ("gh", "g"), ("bh", "b"), ("ph", "f"),
                    ("gaw", "gaon"), ("ksh", "x"), ("dh", "d"), ("sh", "s")):
                # ("g","j"), ("o", "u"), ("i", "e"), ("m", "n"),
                # ("y", "i"), ("v", "w"), ("z","j"),, ("a","")
                i = i.replace(*r)

            newtup.append(''.join(i for i, _ in itertools.groupby(i)))
        return tuple(newtup)


In [3]:
s = IndianNameStemmer()

In [4]:
s.stem([ 'm.sattar', 'BHimrav'])

('satar', 'bim')

In [5]:
edit_distance( 'rajandar', 'rajendra', transpositions=True)

2

# compare names from 2 dataframes

In [6]:
from io import StringIO

myst="""shantanu prabhakar oka, 905034 , 19:44   
sammeer annashaaheb goankar, 905094  , 19:33
paravatibai vittal shelke,  905154 ,   21:56
"""
u_cols=['name', 'my_index', 'current_tm']

myf = StringIO(myst)
import pandas as pd
df = pd.read_csv(StringIO(myst), sep=',', names = u_cols)

In [7]:
df

,name,my_index,current_tm
0,shantanu prabhakar oka,905034,19:44
1,sammeer annashaaheb goankar,905094,19:33
2,paravatibai vittal shelke,905154,21:56


In [8]:
myst="""shantanu34 prabakar oka, 905034 , 19:44   
sammir anashaheb goankar, 905094  , 19:33
parwati vithal bholke,  905154 ,   21:56
anashaheb  gavkar, 905094  , 19:33
parwati vothald shelke,  905154 ,   21:56
"""
u_cols=['name', 'my_index', 'current_tm']

myf = StringIO(myst)
import pandas as pd
df1 = pd.read_csv(StringIO(myst), sep=',', names = u_cols)

In [9]:
df1

,name,my_index,current_tm
0,shantanu34 prabakar oka,905034,19:44
1,sammir anashaheb goankar,905094,19:33
2,parwati vithal bholke,905154,21:56
3,anashaheb gavkar,905094,19:33
4,parwati vothald shelke,905154,21:56


In [10]:
df['updated']=df['name'].apply(str).apply(lambda x: x.split()).apply(s.stem).apply(set).apply(sorted).apply(tuple)

In [11]:
df1['updated1']=df1['name'].apply(str).apply(lambda x: x.split()).apply(s.stem).apply(set).apply(sorted).apply(tuple)

In [12]:
df

,name,my_index,current_tm,updated
0,shantanu prabhakar oka,905034,19:44,"(oka, prabakar, santanu)"
1,sammeer annashaaheb goankar,905094,19:33,"(anasaheb, goankar, samer)"
2,paravatibai vittal shelke,905154,21:56,"(paravati, selke, vital)"


In [13]:
df1

,name,my_index,current_tm,updated1
0,shantanu34 prabakar oka,905034,19:44,"(oka, prabakar, santanu)"
1,sammir anashaheb goankar,905094,19:33,"(anasaheb, goankar, samir)"
2,parwati vithal bholke,905154,21:56,"(bolke, parwati, vital)"
3,anashaheb gavkar,905094,19:33,"(anasaheb, gavkar)"
4,parwati vothald shelke,905154,21:56,"(parwati, selke, votald)"


In [14]:
df.merge(df1, how='outer', left_on=['updated'], right_on=['updated1'], indicator=True)

,name_x,my_index_x,current_tm_x,updated,name_y,my_index_y,current_tm_y,updated1,_merge
0,shantanu prabhakar oka,905034.0,19:44,"(oka, prabakar, santanu)",shantanu34 prabakar oka,905034.0,19:44,"(oka, prabakar, santanu)",both
1,sammeer annashaaheb goankar,905094.0,19:33,"(anasaheb, goankar, samer)",NaN,NaN,NaN,NaN,left_only
2,paravatibai vittal shelke,905154.0,21:56,"(paravati, selke, vital)",NaN,NaN,NaN,NaN,left_only
3,NaN,NaN,NaN,NaN,sammir anashaheb goankar,905094.0,19:33,"(anasaheb, goankar, samir)",right_only
4,NaN,NaN,NaN,NaN,parwati vithal bholke,905154.0,21:56,"(bolke, parwati, vital)",right_only
5,NaN,NaN,NaN,NaN,anashaheb gavkar,905094.0,19:33,"(anasaheb, gavkar)",right_only
6,NaN,NaN,NaN,NaN,parwati vothald shelke,905154.0,21:56,"(parwati, selke, votald)",right_only


In [15]:
mylist=list()
mylist1=list()
for i, rows in df.iterrows():
        for i1, rows1 in df1.iterrows():
            #mylist.append(edit_distance(rows['updated'], rows1['updated1'], substitution_cost=2))
            #print (rows['updated'], rows1['updated1'], edit_distance(rows['updated'], rows1['updated1'], substitution_cost=2))
            s1, s2 = " ".join(list(rows['updated'])), " ".join(list(rows1['updated1']))  
            print (s1, s2)
            tempd=edit_distance(s1,s2, transpositions=True)
            mylist1.append(tempd)
            final=list()
            for i in s1.split():
                mylisti=list()
                for k in s2.split():
                    mylisti.append(edit_distance(i, k, transpositions=True))
                    #print (i,k,  edit_distance(i, k, transpositions=True) )
                final.append(min(mylisti))
            mylist.append(sum(final))


oka prabakar santanu oka prabakar santanu
oka prabakar santanu anasaheb goankar samir
oka prabakar santanu bolke parwati vital
oka prabakar santanu anasaheb gavkar
oka prabakar santanu parwati selke votald
anasaheb goankar samer oka prabakar santanu
anasaheb goankar samer anasaheb goankar samir
anasaheb goankar samer bolke parwati vital
anasaheb goankar samer anasaheb gavkar
anasaheb goankar samer parwati selke votald
paravati selke vital oka prabakar santanu
paravati selke vital anasaheb goankar samir
paravati selke vital bolke parwati vital
paravati selke vital anasaheb gavkar
paravati selke vital parwati selke votald


In [16]:
fdf = pd.merge(df.assign(key=0), df1.assign(key=0), on='key').drop('key', axis=1)

In [17]:
fdf['eud'] = mylist
fdf['eud1'] = mylist1

In [18]:
fdf

,name_x,my_index_x,current_tm_x,updated,name_y,my_index_y,current_tm_y,updated1,eud,eud1
0,shantanu prabhakar oka,905034,19:44,"(oka, prabakar, santanu)",shantanu34 prabakar oka,905034,19:44,"(oka, prabakar, santanu)",0,0
1,shantanu prabhakar oka,905034,19:44,"(oka, prabakar, santanu)",sammir anashaheb goankar,905094,19:33,"(anasaheb, goankar, samir)",13,15
2,shantanu prabhakar oka,905034,19:44,"(oka, prabakar, santanu)",parwati vithal bholke,905154,21:56,"(bolke, parwati, vital)",13,13
3,shantanu prabhakar oka,905034,19:44,"(oka, prabakar, santanu)",anashaheb gavkar,905094,19:33,"(anasaheb, gavkar)",13,14
4,shantanu prabhakar oka,905034,19:44,"(oka, prabakar, santanu)",parwati vothald shelke,905154,21:56,"(parwati, selke, votald)",14,17
5,sammeer annashaaheb goankar,905094,19:33,"(anasaheb, goankar, samer)",shantanu34 prabakar oka,905034,19:44,"(oka, prabakar, santanu)",15,15
6,sammeer annashaaheb goankar,905094,19:33,"(anasaheb, goankar, samer)",sammir anashaheb goankar,905094,19:33,"(anasaheb, goankar, samir)",1,1
7,sammeer annashaaheb goankar,905094,19:33,"(anasaheb, goankar, samer)",parwati vithal bholke,905154,21:56,"(bolke, parwati, vital)",17,18
8,sammeer annashaaheb goankar,905094,19:33,"(anasaheb, goankar, samer)",anashaheb gavkar,905094,19:33,"(anasaheb, gavkar)",6,8
9,sammeer annashaaheb goankar,905094,19:33,"(anasaheb, goankar, samer)",parwati vothald shelke,905154,21:56,"(parwati, selke, votald)",17,19


In [19]:
gdf=fdf.groupby(['name_x'])['eud'].min()

In [20]:
gdf=gdf.reset_index()

In [21]:
gdf

,name_x,eud
0,paravatibai vittal shelke,4
1,sammeer annashaaheb goankar,1
2,shantanu prabhakar oka,0


In [22]:
gdf.merge(fdf, how='left', left_on=['name_x', 'eud'], right_on=['name_x', 'eud'])

,name_x,eud,my_index_x,current_tm_x,updated,name_y,my_index_y,current_tm_y,updated1,eud1
0,paravatibai vittal shelke,4,905154,21:56,"(paravati, selke, vital)",parwati vithal bholke,905154,21:56,"(bolke, parwati, vital)",14
1,paravatibai vittal shelke,4,905154,21:56,"(paravati, selke, vital)",parwati vothald shelke,905154,21:56,"(parwati, selke, votald)",4
2,sammeer annashaaheb goankar,1,905094,19:33,"(anasaheb, goankar, samer)",sammir anashaheb goankar,905094,19:33,"(anasaheb, goankar, samir)",1
3,shantanu prabhakar oka,0,905034,19:44,"(oka, prabakar, santanu)",shantanu34 prabakar oka,905034,19:44,"(oka, prabakar, santanu)",0
